In [1]:
import cv2
import os
import numpy as np

In [2]:
dirname = 'lfwcrop_grey_sample/'
filenames = os.listdir(dirname)

In [3]:
def load_imgs(dirname, filenames):
    imgs = []
    target_names = []
    print len(filenames)
    for fname in filenames:
        if fname.endswith('.pgm'):
            im = cv2.imread(dirname + fname)
            im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
            # print im.shape
            im = np.reshape(im, 64 * 64)
            imgs.append(im)
            target_names.append(' '.join(fname.split('_')[:-1]))
    imgs = np.stack(imgs, axis = 0)
    print imgs.size
    return imgs, target_names

In [4]:
from sklearn.cross_validation import train_test_split
from random import shuffle

shuffle(filenames)

X, target_names = load_imgs(dirname, filenames)

class Counting(object):
    n = 0
    def factory():
        Counting.n += 1
        return Counting.n
    factory = staticmethod(factory)

from collections import defaultdict
names_id = defaultdict(Counting.factory)

y = [names_id[t] for t in target_names] 

7606
31154176


In [5]:
len(target_names)

7606

In [20]:
#X_train, X_test, y_train, y_test = train_test_split(filenames, y, test_size=0.2)
#y_train = load_imgs(dirname, X_train)
#y_test = load_imgs(dirname, X_test)


7606
31154176


# PCA

In [6]:
from sklearn.decomposition import PCA
import numpy as np

In [7]:
def center(faces):
    n_samples, n_features = faces.shape
    # Center the faces
    faces_centered = faces - faces.mean(axis=0)
    faces_centered -= faces_centered.mean(axis=1).reshape(n_samples, -1)
    return faces_centered

faces_centered = center(X)

In [11]:
n_components = 150

In [12]:
from sklearn import decomposition
from time import time

In [13]:
from sklearn.decomposition import RandomizedPCA

# We use a variant of PCA called Randomized PCA for efficiency. It uses stochastic SVD.
print("Extracting the top %d eigenfaces from %d faces" % (n_components, X.shape[0]))
t0 = time()
pca = RandomizedPCA(n_components=n_components, whiten=True).fit(X)
print("done in %0.3fs" % (time() - t0))

eigenfaces = pca.components_.reshape((n_components, 64, 64))

print("Projecting the input data on the eigenfaces orthonormal basis")
t0 = time()
X_pca = pca.transform(faces_centered)
print("done in %0.3fs" % (time() - t0))
print(X_pca.shape)
print(X_pca)

Extracting the top 150 eigenfaces from 7606 faces
done in 2.040s
Projecting the input data on the eigenfaces orthonormal basis
done in 0.334s
(7606, 150)
[[ 5.8567939   0.16690285  1.04884043 ..., -0.76495653  1.1846018
  -0.47364922]
 [ 5.72912698  0.10210857  2.77405136 ..., -0.69152252  0.38172012
  -0.26452762]
 [ 5.83938851  0.10462778  1.55223946 ...,  0.82302809 -0.42707468
   1.19394587]
 ..., 
 [ 5.86341945 -0.84305815  1.17425937 ..., -2.18073399  0.32904926
  -0.3978569 ]
 [ 5.71484353  1.19164204  3.12452724 ...,  1.22536156 -0.52958635
   0.57424227]
 [ 5.81759071 -0.46013476  1.86819473 ..., -0.41502462  0.5434003
   0.89106038]]


In [14]:
import cPickle

with open("pca_images.pkl", "wb") as f:
    cPickle.dump(X_pca, f)
    
with open("pca_person_id.pkl", "wb") as f:
    cPickle.dump(y, f)

In [15]:

with open("pca_filenames.pkl", "wb") as fx:
    cPickle.dump([f for f in filenames if f.endswith('.pgm')], fx)
    
    
with open("pca_person_names.pkl", "wb") as fx:
    cPickle.dump(target_names, fx)

In [26]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_pca, y) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [35]:
im = cv2.imread("fun.jpg")
im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im = cv2.resize(im, (64, 64))
im = np.reshape(im, 64 * 64)

imp = pca.transform([im])
_, idx = model.kneighbors(imp)

for i in idx[0]:
    print target_names[i]
    

Hugo Chavez
Bill Clinton
George W Bush


In [42]:
# Sanity
_, idx = model.kneighbors([X_pca[100]])
idx

array([[ 100, 7454, 5047]])